In [ ]:
import numpy as np
from tqdm import tqdm
import MDAnalysis as mda
#from scipy.spatial.transform import Rotation as R

In [ ]:
from utilities import generate_names, defect_report, insert_group, Hgroup, generate_charges

#### Import the amorphous silica and add hydrogen to dandling oxygen

In [ ]:
u = mda.Universe("silica-deformed.data")
# the next lines are used to complete the universe with atom names, residue names, and redidue id
u.add_TopologyAttr('names')
u.add_TopologyAttr('resname', ['Sil']*u.residues.n_residues)
u.add_TopologyAttr('resid', np.arange(u.residues.n_residues).tolist())
u.atoms.names = generate_names(u.select_atoms("all"))
# detect dandling atoms
O_neighbor = defect_report(u, g_subject="name O", g_neighbor="name Si C",  expected_neighbor = 2, cutoff_distance = 2.5)
# loop on the dandling oxygen and insert hydrogen
for center_id in tqdm(O_neighbor[O_neighbor.T[1] == 1].T[0]):
    center_position = u.select_atoms("name O").positions[u.select_atoms("name O").indices == center_id]
    u = insert_group(u, center_position, center_id, group=Hgroup(), cut_off_energy=0.01, verbose=False)
# correct the charge, name and residue information
u.add_TopologyAttr('resname', ['Sil']*len(u.residues))
u.add_TopologyAttr('resid', np.arange(u.residues.n_residues).tolist())
u.atoms.names = generate_names(u.select_atoms("all"))
u.add_TopologyAttr('charge', generate_charges(u.select_atoms("all"), rescale_charges=True))

#### Write decorated data file

In [ ]:
filename = "decorated.data"
f = open(filename, "w")
s = " "
n = "\n"
f.write("# LAMMPS data file"+n+n)
f.write(str(u.atoms.n_atoms)+s+"atoms"+n)
f.write("3 atom types"+n)
f.write(n)
f.write(str(-u.dimensions[0]/2)+s+str(u.dimensions[0]/2)+s+"xlo xhi"+n)
f.write(str(-u.dimensions[1]/2)+s+str(u.dimensions[1]/2)+s+"ylo yhi"+n)
f.write(str(-u.dimensions[2]/2)+s+str(u.dimensions[2]/2)+s+"zlo zhi"+n)
f.write(n)
f.write("Atoms"+n)
f.write(n)
for id, tp, cp, pos in zip(u.atoms.indices, u.atoms.types, u.atoms.charges, u.atoms.positions):
    f.write(str(id+1)+s+"1"+s+tp+s+str(cp)+s+str(pos[0])+s+str(pos[1])+s+str(pos[2])+n)
f.close()